In [53]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import pandas as pd
from datetime import datetime
from google.oauth2.service_account import Credentials
import gspread
from google.cloud import storage
from datetime import datetime
from pathlib import Path
import asyncio

import subprocess


In [54]:
data = {
    "userid": [
    "10002536", "10008847", "10004679", "10023836",
    "10033752", "10038584", "10044023", "10192623",
    "10165926", "10093976", "10132431"
    ],
    "名前": [
        "キンシュウサイ", "大岡正樹", "池海龍（イケカイリュウ）", "飯島美桜",
        "江原ケイト", "樋口大輔", "加藤順平", "加藤奈津子",
        "松本明子", "石崎卓", "玉野利家"
    ],
    "PW": [
        "Kin20240301", "Hasegawa110", "Chihailong0803", "Cherry2525",
        "Tennis2784!", "46495963¥@Desu", "WCbifd@3NANwXGE", "aDA8a6MWs3XEz_Y",
        "Saikou1234", "Zaitaku2023", "Packen0731t"
    ]
}

user_df = pd.DataFrame(data)

COLUMNS = ["userid","予約番号", "利用日", "時間", "公園・施設", "設備予約", "支払状況"]

In [66]:
URL = "https://kouen.sports.metro.tokyo.lg.jp/web/index.jsp"
BLOCK_TEXT = "しばらく経ってから"
result_lst = []
async def scrape_cancel_list(user_df):
    p = await async_playwright().start()

    context = await p.chromium.launch_persistent_context(
        user_data_dir="./check_reservation",
        channel="chrome",
        headless=False,
    )
    page = await context.new_page()
    # page.on("request", lambda r: print(">>", r.method, r.url))
    # page.on("response", lambda r: print("<<", r.status, r.url))
    for _, row in user_df.iterrows():
        userid = str(row["userid"])
        pw = str(row["PW"])

        print(f"Processing user: {userid}")

        await page.goto(URL)

        # --- wait until not blocked ---
        while True:
            content = await page.content()
            if BLOCK_TEXT not in content:
                break
            print("Blocked. Waiting 5 seconds...")
            await asyncio.sleep(5)
            await page.reload()

        await page.click("text=ログイン")
        await page.fill("#userId", userid)
        await page.fill("#password", pw)
        # await page.wait_for_timeout(1500)
        
        # Login often triggers navigation
        async with page.expect_navigation(wait_until="networkidle"):
            await page.click("#btn-go")

        await page.wait_for_timeout(1500)

        # If login failed, login form will still exist
        if await page.locator("#userId").count():
            result_lst.append([userid, "login_fail", "login_fail", "login_fail", "login_fail", "login_fail", "login_fail"])
            continue

        # 2) Open reservation modal (as you already did)
        await page.click('a[data-target="#modal-reservation-menus"]')
        await page.wait_for_selector("#modal-reservation-menus", state="visible")

        # 3) Trigger the cancel/reservation list action ONCE and wait for navigation (if it happens)
        try:
            async with page.expect_navigation(wait_until="networkidle", timeout=10000):
                await page.evaluate("doAction(document.form1, gRsvWGetCancelRsvDataAction)")
        except PlaywrightTimeoutError:
            # In some flows it may update without a full navigation
            await page.wait_for_load_state("networkidle")

        # 4) If no reservations, tbody won't exist -> skip
        tbody = await page.query_selector("#rsvacceptlist tbody")

        if not tbody:
            result_lst.append([userid, "", "", "", "", "", ""])
        else:
            rows = await page.evaluate("""
            [...document.querySelectorAll('#rsvacceptlist tbody tr')]
            .map(tr => [...tr.querySelectorAll('td')]
                .slice(0, 6)
                .map(td => td.innerText.trim()))
            .filter(r => r.length === 6)
            """)
            result_lst.extend([[userid, *r] for r in rows])

        await page.goto(URL)
    
    await context.close()
    await p.stop()
    return pd.DataFrame(result_lst, columns=COLUMNS)
final_df = await scrape_cancel_list(user_df)


Processing user: 10002536
Processing user: 10008847
Processing user: 10004679
Processing user: 10023836
Processing user: 10033752
Processing user: 10038584
Processing user: 10044023
Processing user: 10192623
Processing user: 10165926
Processing user: 10093976
Processing user: 10132431


In [71]:
local_folder = Path.home() / "workspace" / "熊猫カンパニー" / "reservation"
remote_folder = "googledrive:熊猫カンパニー/reservation"

today_str = datetime.today().strftime("%Y%m%d")
file_path = local_folder / f"reservations_{today_str}.xlsx"

final_df.to_excel(file_path, index=False)

reservation_hist_df = final_df[final_df["設備予約"]=="あり"]
reservation_hist_df = reservation_hist_df.replace(r"\s*\n\s*", " ", regex=True)
reservation_hist_df["時間"] = reservation_hist_df["時間"].str.replace(r"\s*～.*", "", regex=True)

reservation_hist_df = user_df[["userid","名前"]].merge(reservation_hist_df, on=["userid"], how="right").sort_values(by=["利用日"], ascending=False)
reservation_hist_df["利用日"] = (
    reservation_hist_df["利用日"]
    .str.replace(r"\)\s*.*", ")", regex=True)
)
reservation_hist_df["利用日"] = (
    reservation_hist_df["利用日"]
    .str.replace(r"(\d{1,2})月(\d{1,2})日", 
                 lambda m: f"{int(m.group(1)):02d}月{int(m.group(2)):02d}日",
                 regex=True)
)


reservation_hist_df.drop(columns=["設備予約","支払状況"], inplace=True)
reservation_hist_df.rename(columns={"時間": "開始"}, inplace=True)
reservation_hist_df = pd.concat([pd.read_excel(local_folder / f"予約管理.xlsx",sheet_name="reservations_2026")   , reservation_hist_df], ignore_index=True)
reservation_hist_df["予約番号"] = reservation_hist_df["予約番号"].astype(str)
reservation_hist_df["利用日"].sort_values(ascending=False, inplace=True)
reservation_hist_df.drop_duplicates(subset=["予約番号"],keep="first", inplace=True)
reservation_hist_df.to_excel(local_folder / f"予約管理.xlsx", sheet_name=f"reservations_2026", index=False    )
reservation_hist_df.to_excel(local_folder / "予約管理_backup" / f"予約管理_{today_str}.xlsx", sheet_name=f"reservations_2026", index=False)


subprocess.run(
    ["rclone", "copy", local_folder, remote_folder, "-P"],
    check=True
)

subprocess.run(
    ["paplay", "/usr/share/sounds/freedesktop/stereo/complete.oga"],
    check=False
)


Transferred:   	   12.537 KiB / 12.537 KiB, 100%, 0 B/s, ETA -
Checks:                 4 / 4, 100%
Transferred:            0 / 2, 0%
Elapsed time:         1.3s
Transferring:
 *                                     予約管理.xlsx:100% /7.075Ki, 0/s, -
 *                    reservations_20260217.xlsx:100% /5.462Ki, 0/s, -Transferred:   	   19.612 KiB / 19.612 KiB, 100%, 0 B/s, ETA -
Checks:                 5 / 5, 100%
Transferred:            0 / 3, 0%
Elapsed time:         1.8s
Transferring:
 *                                     予約管理.xlsx:100% /7.075Ki, 0/s, -
 *                    reservations_20260217.xlsx:100% /5.462Ki, 0/s, -
 *                予約管理_backup/予約管理_20260217.xlsx:100% /7.075Ki, Transferred:   	   19.612 KiB / 19.612 KiB, 100%, 19.606 KiB/s, ETA 0s
Checks:                 5 / 5, 100%
Transferred:            0 / 3, 0%
Elapsed time:         2.3s
Transferring:
 *                                     予約管理.xlsx:100% /7.075Ki, 7.072Ki/
 *                    reservations_20260217.xlsx:1

CompletedProcess(args=['paplay', '/usr/share/sounds/freedesktop/stereo/complete.oga'], returncode=0)